In [ ]:
from pyspark.sql import SparkSession
from src.data_loader import load_shodan_data
from src.analysis import process_shodan_data
from src.enrichment import get_top3_domains
from src.hunter_api import create_domain_index, fetch_and_store_hunter_data
from src.report_generator import generate_top_report

spark = SparkSession.builder.appName("ShodanPulseSecure").getOrCreate()

df = load_shodan_data("data/ivanti_data.json")
processed_df = process_shodan_data(df)
top3_domains = get_top3_domains(processed_df)

create_domain_index()
fetch_and_store_hunter_data(top3_domains, enrich=True, force=False)

report = generate_top_report(processed_df, top_n=3, save_to_csv=True)
display(report)
